# Part 3 — Data Quality

## Links
- **Live updated app:** https://ind320-project-work-nonewthing.streamlit.app/
- **Repo:** https://github.com/TaoM29/IND320-dashboard-basics

## Plan for Part 3
1. 

# AI Usage

Used an ChatGPT 5 as a coding and troubleshooting partner. It helped me:

# Work Log

In [5]:
import pandas as pd

# Representative cities for NO1–NO5 and their approximate city-center coordinates
rows = [

    # price_area, city, longitude, latitude
    ("NO1", "Oslo",         10.7461, 59.9127),
    ("NO2", "Kristiansand",  7.9956, 58.1467),
    ("NO3", "Trondheim",    10.3951, 63.4305),
    ("NO4", "Tromsø",       18.9551, 69.6489),
    ("NO5", "Bergen",        5.3242, 60.3930),
    
]

df_price_areas = pd.DataFrame(
    rows, columns=["price_area", "city", "longitude", "latitude"]
)

df_price_areas


,price_area,city,longitude,latitude
0,NO1,Oslo,10.7461,59.9127
1,NO2,Kristiansand,7.9956,58.1467
2,NO3,Trondheim,10.3951,63.4305
3,NO4,Tromsø,18.9551,69.6489
4,NO5,Bergen,5.3242,60.3930


In [16]:
import pandas as pd
import requests
from datetime import date

# Columns from Part 1 CSV
_CSV_COLUMNS = [
    "time",
    "temperature_2m (°C)",
    "precipitation (mm)",
    "wind_speed_10m (m/s)",
    "wind_gusts_10m (m/s)",
    "wind_direction_10m (°)",
]

# Map CSV columns -> Open-Meteo ERA5 hourly variable names
_VAR_MAP = {
    "temperature_2m (°C)": "temperature_2m",
    "precipitation (mm)": "precipitation",
    "wind_speed_10m (m/s)": "windspeed_10m",
    "wind_gusts_10m (m/s)": "windgusts_10m",
    "wind_direction_10m (°)": "winddirection_10m",
}

def download_era5_to_df(longitude: float, latitude: float, year: int, timezone: str = "Europe/Oslo") -> pd.DataFrame:
    """
    Download ERA5 historical reanalysis (Open-Meteo archive API) for a single location and year.
    Returns a DataFrame with the SAME columns as our Part 1 CSV.
    """

    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    if year == date.today().year:
        # Ensure we don't request future dates if running mid-year
        end_date = min(end_date, date.today().isoformat())

    url = "https://archive-api.open-meteo.com/v1/era5"
    hourly_vars = list(_VAR_MAP.values())

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": hourly_vars,
        "timezone": timezone,
    }

    r = requests.get(url, params=params, timeout=60)
    r.raise_for_status()
    data = r.json()

    # Parse hourly block
    hourly = data.get("hourly", {})
    times = hourly.get("time", [])

    # Build DataFrame with CSV-matching columns/order
    df = pd.DataFrame({"time": pd.to_datetime(times)})

    for csv_col, api_var in _VAR_MAP.items():
        df[csv_col] = hourly.get(api_var, [None] * len(df))

    # Ensure monotonic time index and set dtype
    df = df.sort_values("time").reset_index(drop=True)
    return df


# Apply the function to Bergen for the year 2019.
bergen = download_era5_to_df(longitude=5.3242, latitude=60.3930, year=2019)
bergen.head()

 

,time,temperature_2m (°C),precipitation (mm),wind_speed_10m (m/s),wind_gusts_10m (m/s),wind_direction_10m (°)
0,2019-01-01 00:00:00,5.7,0.7,37.0,99.7,263
1,2019-01-01 01:00:00,5.8,0.2,41.0,107.3,278
2,2019-01-01 02:00:00,6.1,0.7,42.0,112.0,286
3,2019-01-01 03:00:00,6.3,0.5,40.9,105.8,298
4,2019-01-01 04:00:00,5.8,1.1,41.2,110.2,315
